<a href="https://colab.research.google.com/github/thiagofuruchima/weratedogs_dsnd/blob/main/wrangle_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd
import json
import time
import tweepy
from tweepy import TweepError
from pandas.io.json import json_normalize

## Data Wrangling Process
<ol>
    <li><strong> Gather; </strong></li>
    <li><strong> Assess; </strong></li>
    <li><strong> Clean. </strong></li>
</ol>

### 1. Gather the Data

#### twitter-archive-enhanced.csv

In [115]:
df_twitter_archive = pd.read_csv("twitter-archive-enhanced.csv", sep=',')

In [4]:
df_twitter_archive.shape

(2356, 17)

In [5]:
df_twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [6]:
df_twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### image-predictions.tsv

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

In [ ]:
response = requests.get(url)
if (response.status_code==200):
    with open('image-predictions.tsv', 'wb') as file:
        file.write(response.content)

In [7]:
df_image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')

In [8]:
df_image_predictions.shape

(2075, 12)

In [9]:
df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [10]:
df_image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### tweet_json.txt

In [ ]:
import twitter_config

In [ ]:
twitter_config.access_token_secret

'cDC3yasRWXWtlROHbM4fUzvzWKJlkbhZgVqEM2BFGrAjv'

In [ ]:
auth = tweepy.OAuthHandler(twitter_config.consumer_key, twitter_config.consumer_secret)

auth.set_access_token(twitter_config.access_token, twitter_config.access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# get all unique tweet_ids from the twitter-archive-enhanced.csv dataframe
tweet_ids = set(df_twitter_archive['tweet_id'].unique())

with open('all_tweet.json', 'wt') as file:
    for tweet_id in tweet_ids:
        
        print ('Reading tweet {}'.format(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, file)
            file.write('\n')
            time.sleep(1)
        except TweepError as e:
            print ('Error reading tweet {}'.format(tweet_id))
            print(e)

In [14]:
list_df = []
with open('all_tweet.json') as file:
  for line in file:
    df_tweet = pd.DataFrame(pd.json_normalize(json.loads(line)))
    list_df.append(df_tweet)

df_all_tweet_info = pd.concat(list_df, axis=0)

In [15]:
df_all_tweet_info.head()

,created_at,id,id_str,full_text,truncated,display_text_range,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,entities.hashtags,entities.symbols,entities.user_mentions,entities.urls,entities.media,extended_entities.media,user.id,user.id_str,user.name,user.screen_name,user.location,user.description,user.url,user.entities.url.urls,user.entities.description.urls,user.protected,...,quoted_status.user.profile_use_background_image,quoted_status.user.has_extended_profile,quoted_status.user.default_profile,quoted_status.user.default_profile_image,quoted_status.user.following,quoted_status.user.follow_request_sent,quoted_status.user.notifications,quoted_status.user.translator_type,quoted_status.geo,quoted_status.coordinates,quoted_status.place,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.possibly_sensitive,quoted_status.possibly_sensitive_appealable,quoted_status.lang,retweeted_status.place.id,retweeted_status.place.url,retweeted_status.place.place_type,retweeted_status.place.name,retweeted_status.place.full_name,retweeted_status.place.country_code,retweeted_status.place.country,retweeted_status.place.contained_within,retweeted_status.place.bounding_box.type,retweeted_status.place.bounding_box.coordinates,place.id,place.url,place.place_type,place.name,place.full_name,place.country_code,place.country,place.contained_within,place.bounding_box.type,place.bounding_box.coordinates
0,Mon Jul 31 00:18:03 +0000 2017,891815181378084864,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,False,"[0, 121]","<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,None,None,None,None,None,False,3632,22828,False,False,False,False,en,[],[],[],[],"[{'id': 891815175371796480, 'id_str': '8918151...","[{'id': 891815175371796480, 'id_str': '8918151...",4196983835,4196983835,WeRateDogs®,dog_rates,links and things ➜,Your Only Source For Professional Dog Ratings ...,https://t.co/6ytGi24QCk,"[{'url': 'https://t.co/6ytGi24QCk', 'expanded_...",[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Tue Aug 01 16:23:56 +0000 2017,892420643555336193,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,False,"[0, 85]","<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,None,None,None,None,None,False,7372,35072,False,False,False,False,en,[],[],[],[],"[{'id': 892420639486877696, 'id_str': '8924206...","[{'id': 892420639486877696, 'id_str': '8924206...",4196983835,4196983835,WeRateDogs®,dog_rates,links and things ➜,Your Only Source For Professional Dog Ratings ...,https://t.co/6ytGi24QCk,"[{'url': 'https://t.co/6ytGi24QCk', 'expanded_...",[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Sat Jul 29 16:00:24 +0000 2017,891327558926688256,891327558926688256,This is Franklin. He would like you to stop ca...,False,"[0, 138]","<a href=""http://twitter.com/download/iphone"" r...",None,None,None,None,None,None,None,None,None,False,8137,36622,False,False,False,False,en,"[{'text': 'BarkWeek', 'indices': [129, 138]}]",[],[],[],"[{'id': 891327551943041024, 'id_str': '8913275...","[{'id': 891327551943041024, 'id_str': '8913275...",4196983835,4196983835,WeRateDogs®,dog_rates,links and things ➜,Your Only Source For Professional Dog Ratings ...,https://t.co/6ytGi24QCk,"[{'url': 'https://t.co/6ytGi24QCk', 'expanded_...",[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [16]:
df_retweet_favorite_info = df_all_tweet_info[['id','retweet_count', 'favorite_count', 'created_at']]

In [17]:
df_retweet_favorite_info.shape

(2331, 4)

In [18]:
df_retweet_favorite_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2331 entries, 0 to 0
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              2331 non-null   int64 
 1   retweet_count   2331 non-null   int64 
 2   favorite_count  2331 non-null   int64 
 3   created_at      2331 non-null   object
dtypes: int64(3), object(1)
memory usage: 91.1+ KB


In [19]:
df_retweet_favorite_info.head()

,id,retweet_count,favorite_count,created_at
0,891815181378084864,3632,22828,Mon Jul 31 00:18:03 +0000 2017
0,892420643555336193,7372,35072,Tue Aug 01 16:23:56 +0000 2017
0,891327558926688256,8137,36622,Sat Jul 29 16:00:24 +0000 2017
0,891689557279858688,7550,38346,Sun Jul 30 15:58:51 +0000 2017
0,890971913173991426,1764,10743,Fri Jul 28 16:27:12 +0000 2017


## 2. Assess

After gathering each of the above pieces of data, assess them visually and programmatically for quality and tidiness issues. Detect and document at least eight (8) quality issues and two (2) tidiness issues in your wrangle_act.ipynb Jupyter Notebook. To meet specifications, the issues that satisfy the Project Motivation (see the Key Points header on the previous page) must be assessed.



#### Quality Issues

In [94]:
df_twitter_archive['name'].value_counts()

None        745
a            55
Charlie      12
Cooper       11
Lucy         11
           ... 
Jeffri        1
Hero          1
Beckham       1
Maya          1
Harrison      1
Name: name, Length: 957, dtype: int64

In [95]:
df_twitter_archive[df_twitter_archive['name'].str.len() < 3]['name'].value_counts()

a     55
Bo     9
an     7
Jo     1
Mo     1
by     1
Ed     1
my     1
Al     1
O      1
JD     1
Name: name, dtype: int64

> 1. We found some problems in dog names such as: 'None', 'a', 'an', 'all' and 'my'.

> 2. After visual inspecting we found the following problems in rating_rumerator:

<ul>
<li>
45 - This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948
</li>
<li>
784 - RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…
</li>
<li>
1068 - After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ
</li>
</ul>

> 3. After visual inspecting, we found the following problems in rating_denominator:
<ul>
<li>
342 - @docmisterio account started on 11/15/15
</li>
<li>
784 - RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…
</li>
<li>
1068 - After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ
</li>
<li>
1662 - This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5
</li>
</ul>

In [113]:
df_twitter_archive[df_twitter_archive[['retweeted_status_id', 'retweeted_status_user_id',	'retweeted_status_timestamp']].notna().any(axis=1)].shape

(181, 17)

> 4. Programatically, we identified 181 <strong>retweets</strong> that should be excluded from the database.

In [114]:
df_twitter_archive[df_twitter_archive[['in_reply_to_status_id', 'in_reply_to_user_id']].notna().any(axis=1)].shape

(78, 17)

> 5. Programatically, we identified 78 <strong>replies</strong> that should be excluded from the database.

In [116]:
df_twitter_archive['timestamp'].dtype

dtype('O')

> 6. Programatically, we identified that the timestamp columns is with incorrect data type.

In [111]:
df_twitter_archive['tweet_id'].isin(list(df_image_predictions['tweet_id'])).value_counts()

True     2075
False     281
Name: tweet_id, dtype: int64

> 7. Programatically, we identified that 281 observations don't have corresponding image.

In [103]:
df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,id,retweet_count,favorite_count,created_at
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1.0,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False,8.924206e+17,7372.0,35072.0,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1.0,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True,8.921774e+17,5494.0,30366.0,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1.0,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,8.918152e+17,3632.0,22828.0,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1.0,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,8.916896e+17,7550.0,38346.0,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2.0,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,8.913276e+17,8137.0,36622.0,Sat Jul 29 16:00:24 +0000 2017


> 8. The columns 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id',	'retweeted_status_user_id', 'retweeted_status_timestamp' are not relevant to our analysis and should be dropped.

#### Tidyness Issues

1. All three DataFrames hold information about the same entity, therefore, it should be considered as a single DataFrame.
2. The dog types are in four different columns, but the information could in fact be in only one.

## Clean

In [156]:
# merge the three DataFrames
_ = pd.merge(df_twitter_archive, df_image_predictions, on='tweet_id', how='left')
df = pd.merge(_, df_retweet_favorite_info, left_on='tweet_id', right_on='id', how='left')

In [157]:
# extract the stage from the text
df['stage'] = df['text'].str.extract('(doggo|floofer|pupper|puppo)', expand = True)

In [158]:
# drop the separate stage columns
df.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

In [ ]:
df_twitter_archive[df_twitter_archive[['retweeted_status_id', 'retweeted_status_user_id',	'retweeted_status_timestamp']].notna().any(axis=1)].shape

In [169]:
# drop replier values
idx = df[df[['in_reply_to_status_id', 'in_reply_to_user_id']].notna().any(axis=1)].index
df.drop(idx, inplace=True)

In [170]:
# drop the unecessary replier columns
df.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], axis=1, inplace=True)

In [166]:
# drop retweet values
idx = df[df[['retweeted_status_id', 'retweeted_status_user_id',	'retweeted_status_timestamp']].notna().any(axis=1)].index
df.drop(idx, inplace=True)

In [167]:
# drop the unecessary retweet columns
df.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)

In [155]:
# convert the timestamp column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [185]:
# drop duplicated id column
df.drop(['id'], axis='columns', inplace=True)

In [175]:
# Fix data issues: 13/10 to 13.5/10
idx = df['text'].str.contains('This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10')
df.loc[idx, ['rating_numerator', 'rating_denominator']] = [13.5, 10]

In [178]:
# Fix data issues: 9/11 to 14/10
idx = df['text'].str.contains('After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP')
df.loc[idx, ['rating_numerator', 'rating_denominator']] = [14, 10]

In [181]:
# Fix data issues: 7/11 to 10/10
idx = df['text'].str.contains('This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10')
df.loc[idx, ['rating_numerator', 'rating_denominator']] = [10, 10]

In [183]:
df.to_csv("twitter_archive_master.csv")

### EDA

In [186]:
df.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,favorite_count,created_at,stage
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13.0,10,Phineas,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1.0,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False,7372.0,35072.0,Tue Aug 01 16:23:56 +0000 2017,NaN
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13.0,10,Tilly,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1.0,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True,5494.0,30366.0,Tue Aug 01 00:17:27 +0000 2017,NaN
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12.0,10,Archie,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1.0,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,3632.0,22828.0,Mon Jul 31 00:18:03 +0000 2017,NaN
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13.0,10,Darla,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1.0,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,7550.0,38346.0,Sun Jul 30 15:58:51 +0000 2017,NaN
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12.0,10,Franklin,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2.0,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,8137.0,36622.0,Sat Jul 29 16:00:24 +0000 2017,NaN
